In [3]:
import __init__
import vsrl_utils as vu
import numpy as np
import copy
import os
import pickle
import math

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

vcoco_all = vu.load_vcoco('vcoco_train')
classes = [x['action_name'] for x in vcoco_all]
action_dict = {}
role_dict = {}
for i, x in enumerate(vcoco_all):
    # print '{:>20s}'.format(x['action_name']), x['role_name']
    action_dict[i+1] = x['action_name']
    role_dict[i+1] = x['role_name']
new_dict = {27:7, 28:8, 29:15}
print action_dict
print role_dict

{1: 'hold', 2: 'stand', 3: 'sit', 4: 'ride', 5: 'walk', 6: 'look', 7: 'hit', 8: 'eat', 9: 'jump', 10: 'lay', 11: 'talk_on_phone', 12: 'carry', 13: 'throw', 14: 'catch', 15: 'cut', 16: 'run', 17: 'work_on_computer', 18: 'ski', 19: 'surf', 20: 'skateboard', 21: 'smile', 22: 'drink', 23: 'kick', 24: 'point', 25: 'read', 26: 'snowboard'}
{1: ['agent', 'obj'], 2: ['agent'], 3: ['agent', 'instr'], 4: ['agent', 'instr'], 5: ['agent'], 6: ['agent', 'obj'], 7: ['agent', 'instr', 'obj'], 8: ['agent', 'obj', 'instr'], 9: ['agent', 'instr'], 10: ['agent', 'instr'], 11: ['agent', 'instr'], 12: ['agent', 'obj'], 13: ['agent', 'obj'], 14: ['agent', 'obj'], 15: ['agent', 'instr', 'obj'], 16: ['agent'], 17: ['agent', 'instr'], 18: ['agent', 'instr'], 19: ['agent', 'instr'], 20: ['agent', 'instr'], 21: ['agent'], 22: ['agent', 'instr'], 23: ['agent', 'obj'], 24: ['agent', 'instr'], 25: ['agent', 'obj'], 26: ['agent', 'instr']}


In [2]:
sqrt = math.sqrt
def abboxdist(bbox1, bbox2):
    x1, y1, x2, y2 = bbox1
    xx1, yy1, xx2, yy2 = bbox2
    dist = sqrt((x1-xx1) ** 2 + (y1-yy1) ** 2) + sqrt((x2-xx2) ** 2 + (y2-yy2) ** 2)
    return dist

In [3]:
result_file = 'results.txt.all'
assert(os.path.exists(result_file))
prediction_list = []
pred_dict = {}
with open(result_file, 'r') as f:
    for line in f:
        line2 = line[:-1]
        elements = line2.split(' ')
        if len(elements) == 12:
            imgid = elements[0].rsplit('/')[-1]
            imgid = int(imgid.split('.')[0])
            person_box = [float(elements[1]), float(elements[2]), float(elements[3]), float(elements[4])]
            score = float(elements[-1])
            if score < 0.05:
                continue
            found = False
            for agent_ins in prediction_list:
                if imgid == agent_ins['image_id'] and abboxdist(agent_ins['person_box'], person_box) < 1:
                    new_pred_dict = agent_ins
                    found = True
                    continue
            if not found:
                new_pred_dict = copy.deepcopy(pred_dict)
                for aid in range(1,27):
                    an = action_dict[aid]
                    for role in role_dict[aid]:
                        k = an + '_' + role
                        if k not in new_pred_dict:
                            if role == 'agent':
                                new_pred_dict[k] = 0
                            else:
                                new_pred_dict[k] = [0, 0, 0, 0, 0]
            obj_box = [float(elements[6]), float(elements[7]), float(elements[8]), float(elements[9]), score]
            action_kind = int(elements[-2])
            rn = 1
            if action_kind > 26:
                rn =  2
                action_kind = new_dict[action_kind]
            keyone = action_dict[action_kind] + '_agent'
            if action_kind > 26 or len(role_dict[action_kind]) > 1:
                keytwo = action_dict[action_kind] + '_' + role_dict[action_kind][rn]
                if new_pred_dict[keytwo][4] > score:
                    #print new_pred_dict[keytwo], obj_box
                    continue
                new_pred_dict[keytwo] = obj_box
            elif score > 0.1:
                print action_dict[action_kind], score
            new_pred_dict['person_box'] = person_box
            new_pred_dict[keyone] = score
            new_pred_dict['image_id'] = imgid
            if not found:
                prediction_list.append(new_pred_dict)
print new_pred_dict
print len(prediction_list)

{'stand_agent': 0, 'look_agent': 0, 'cut_instr': [0, 0, 0, 0, 0], 'lay_agent': 0, 'snowboard_instr': [0, 0, 0, 0, 0], 'cut_obj': [0, 0, 0, 0, 0], 'point_agent': 0, 'surf_instr': [154.5685815148765, 205.7037966787962, 409.6667096408679, 242.28581269533976, 0.08622593653885693], 'drink_agent': 0, 'skateboard_instr': [0, 0, 0, 0, 0], 'kick_agent': 0, 'smile_agent': 0, 'kick_obj': [0, 0, 0, 0, 0], 'run_agent': 0, 'talk_on_phone_agent': 0, 'catch_obj': [0, 0, 0, 0, 0], 'eat_obj': [0, 0, 0, 0, 0], 'hit_agent': 0, 'cut_agent': 0, 'sit_instr': [0, 0, 0, 0, 0], 'throw_obj': [0, 0, 0, 0, 0], 'eat_instr': [0, 0, 0, 0, 0], 'look_obj': [0, 0, 0, 0, 0], 'surf_agent': 0.08622593653885693, 'hit_instr': [0, 0, 0, 0, 0], 'hit_obj': [0, 0, 0, 0, 0], 'sit_agent': 0, 'skateboard_agent': 0, 'carry_obj': [0, 0, 0, 0, 0], 'read_obj': [0, 0, 0, 0, 0], 'ride_instr': [0, 0, 0, 0, 0], 'jump_agent': 0, 'walk_agent': 0, 'catch_agent': 0, 'work_on_computer_agent': 0, 'read_agent': 0, 'ski_agent': 0, 'throw_agent': 0

In [4]:
for i in range(20):
    print prediction_list[i]

{'stand_agent': 0, 'look_agent': 0, 'cut_instr': [0, 0, 0, 0, 0], 'lay_agent': 0, 'snowboard_instr': [0, 0, 0, 0, 0], 'cut_obj': [0, 0, 0, 0, 0], 'point_agent': 0, 'surf_instr': [0, 0, 0, 0, 0], 'drink_agent': 0, 'skateboard_instr': [0, 0, 0, 0, 0], 'kick_agent': 0, 'smile_agent': 0, 'kick_obj': [0, 0, 0, 0, 0], 'run_agent': 0, 'talk_on_phone_agent': 0, 'catch_obj': [0, 0, 0, 0, 0], 'eat_obj': [480.36119543539434, 240.15855860170328, 515.0798607214077, 271.3900816642428, 0.06467407860350995], 'hit_agent': 0, 'cut_agent': 0, 'sit_instr': [0, 0, 0, 0, 0], 'throw_obj': [0, 0, 0, 0, 0], 'eat_instr': [0, 0, 0, 0, 0], 'look_obj': [0, 0, 0, 0, 0], 'surf_agent': 0, 'hit_instr': [0, 0, 0, 0, 0], 'hit_obj': [0, 0, 0, 0, 0], 'sit_agent': 0, 'skateboard_agent': 0, 'carry_obj': [0, 0, 0, 0, 0], 'read_obj': [0, 0, 0, 0, 0], 'ride_instr': [0, 0, 0, 0, 0], 'jump_agent': 0, 'walk_agent': 0, 'catch_agent': 0, 'work_on_computer_agent': 0, 'read_agent': 0, 'ski_agent': 0, 'throw_agent': 0, 'snowboard_agen

In [5]:
pickle_out = open("detections.pkl","wb")
pickle.dump(prediction_list, pickle_out)
pickle_out.close()

In [6]:
from vsrl_eval import VCOCOeval

vsrl_annot_file = 'data/vcoco/vcoco_val.json'
coco_file = 'data/instances_vcoco_all_2017.json'
split_file = 'data/splits/vcoco_val.ids'
det_file = 'detections.pkl'
vcocoeval = VCOCOeval(vsrl_annot_file, coco_file, split_file)


loading annotations into memory...
Done (t=2.65s)
creating index...
index created!
loading vcoco annotations...


In [7]:
fp, fn =vcocoeval._do_eval(det_file, ovr_thresh=0.5)

---------Reporting Role AP (%)------------------
               hold-obj: AP = 19.13 (#pos = 2163)
              sit-instr: AP = 14.95 (#pos = 1054)
             ride-instr: AP = 41.39 (#pos = 236)
               look-obj: AP = 18.96 (#pos = 2055)
              hit-instr: AP = 43.52 (#pos = 203)
                hit-obj: AP = 18.06 (#pos = 203)
                eat-obj: AP = 17.31 (#pos = 376)
              eat-instr: AP = 3.64 (#pos = 376)
             jump-instr: AP = 48.39 (#pos = 381)
              lay-instr: AP = 13.56 (#pos = 228)
    talk_on_phone-instr: AP = 17.57 (#pos = 187)
              carry-obj: AP = 22.57 (#pos = 262)
              throw-obj: AP = 20.90 (#pos = 141)
              catch-obj: AP = 21.30 (#pos = 161)
              cut-instr: AP = 5.11 (#pos = 173)
                cut-obj: AP = 9.85 (#pos = 173)
 work_on_computer-instr: AP = 32.97 (#pos = 248)
              ski-instr: AP = 27.71 (#pos = 310)
             surf-instr: AP = 54.22 (#pos = 259)
       skateboard-in

In [8]:
pickle_out = open("fp_fn_samples.pkl","wb")
pickle.dump([fp, fn], pickle_out)
pickle_out.close()

In [9]:
print type(fp[0])

<type 'list'>


In [10]:
print type(fp)
for i in fp[-1]:
    print type(i)
for j in fn[0]:
    print type(j)

<type 'list'>
<type 'int'>
<type 'list'>
<type 'list'>
<type 'str'>
<type 'float'>
<type 'int'>
<type 'list'>
<type 'list'>
<type 'str'>
<type 'float'>


In [11]:
a = [0, 3]
print type(a[0])
a = float(a)
print type(a[0])

<type 'int'>


TypeError: float() argument must be a string or a number